In [2]:
import numpy as np
from theano import *
import theano.tensor as T

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [116]:
# Network Layout
features_count = 3
hidden_count = 2
num_layers = 2


x = T.ivector('x')
w1 = theano.shared(numpy.random.randn(features_count, hidden_count), name="w1")
b1 = theano.shared(numpy.random.randn(hidden_count), name="b1")
z1 = x.dot(w1) + b1
a1 = z1 ** 1
w2 = theano.shared(numpy.random.randn(hidden_count ,features_count), name="w2")
b2 = theano.shared(numpy.random.randn(features_count), name="b2")
z2 = a1.dot(w2) + b2
a2 = z2 ** 1

feedforward = function([x], a2, name='feedforward')

In [118]:
# Training Function
eta = theano.shared(0.1,name='eta')
output = T.dvector('output')
cost_graph = ((a2 - x)**2).sum()
compute_cost = function([x], cost_graph, name='compute_cost')
#compute_avg_cost = theano.scan(fn=compute_cost, sequences=[X], outputs_info=None)
def compute_avg_cost(dataset):
    avg_cost = 0
    for r in dataset:
        avg_cost += compute_cost(r) / dataset.shape[1]
    return avg_cost

grad_w2, grad_b2, grad_a_L = T.grad(cost_graph, [w2,b2,a2])
#pp(grad)
#compute_gradient = function([x], cost_grad, name='compute_gradient')

grad_a_L_wtrt_z_L = T.jacobian(expression=a2, wrt=z2).diagonal()
error_L = grad_a_L * grad_a_L_wtrt_z_L
grad_a_l_wtrt_t_l = T.jacobian(expression=a1, wrt=z1).diagonal()
error_l = (w2.dot(error_L)) * grad_a_l_wtrt_t_l
x_as_matrix = x.reshape((features_count, 1)) # transpose and shape to (k x 1) so that matrix multiplication is possible
grad_w1 =  x_as_matrix * error_l #theano.scan(error_l.dot, sequences=x)# error_l.dot(x)
grad_b1 = error_l

delta_w2 = -eta * (grad_w2) 
delta_b2 = -eta * (grad_b2)
delta_w1 = -eta * (grad_w1)
delta_b1 = -eta * (grad_b1)
print('grad_w2',function([x],grad_w2)([1,2,3]))
print('grad_b2',function([x],grad_b2)([1,2,3]))
print('delta_w2',function([x],delta_w2)([1,2,3]))
print('delta_b2',function([x],delta_b2)([1,2,3]))
print('delta_w1',function([x],delta_w1)([1,2,3]))
print('delta_b1',function([x],delta_b1)([1,2,3]))
print('a2',function([x],a2)([1,2,3]))
print('grad_a2', function([x],grad_a_L)([1,2,3]))
print('jacob_L', function([x],grad_a_L_wtrt_z_L)([1,2,3]))
print('error_L', function([x],error_L)([1,2,3]))
print('jacob_l', function([x],grad_a_L_wtrt_z_L)([1,2,3]))
print('w2.dot(error_L)', function([x],(w2.dot(error_L)))([1,2,3]))
print('error_l', function([x],error_l)([1,2,3]))
print('w1', w1.get_value())
print('x_as_matrix', function([x],x_as_matrix)([1,2,3]))
print('grad_w1', function([x],grad_w1)([1,2,3]))
print('cost', compute_cost([1,2,3]))
update_L = function([x], updates=[(w2, w2 + delta_w2), (b2, b2 + delta_b2)])
update_l = function([x], updates=[(w1, w1 + delta_w1), (b1, b1 + delta_b1)])
update_all = function([x], updates=[(w1, w1 + delta_w1), (b1, b1 + delta_b1), (w2, w2 + delta_w2), (b2, b2 + delta_b2)])

grad_w2 [[ 15.37928254   3.21304911  13.28571542]
 [ 24.73194641   5.16701337  21.36521005]]
grad_b2 [-4.71663402  2.59675671  0.19769499]
delta_w2 [[-1.53792825 -0.32130491 -1.32857154]
 [-2.47319464 -0.51670134 -2.136521  ]]
delta_b2 [ 0.4716634  -0.25967567 -0.0197695 ]
delta_w1 [[ 0.47125737  0.24589055]
 [ 0.94251473  0.49178109]
 [ 1.4137721   0.73767164]]
delta_b1 [ 0.47125737  0.24589055]
a2 [-2.35831701  1.29837836  0.09884749]
grad_a2 [-4.71663402  2.59675671  0.19769499]
jacob_L [ 1.  1.  1.]
error_L [-6.71663402 -1.40324329 -5.80230501]
jacob_l [ 1.  1.  1.]
w2.dot(error_L) [-4.71257365 -2.45890547]
error_l [-4.71257365 -2.45890547]
w1 [[ 0.3829944   1.5601257 ]
 [ 0.47455594 -2.10192141]
 [-1.04529492 -0.6261918 ]]
x_as_matrix [[1]
 [2]
 [3]]
grad_w1 [[ -4.71257365  -2.45890547]
 [ -9.42514731  -4.91781093]
 [-14.13772096  -7.3767164 ]]
cost 7.257216307153413


In [122]:
# Run training
data = np.asarray([[1,2,3],[2,3,4],[5,6,7]])
iteration_count = 20
eta.set_value(0.001)
# iterations
cost_beginning = compute_avg_cost(data)
for run in range(0, iteration_count):
    print('iteration', run)
    # epoch
    for row in data:
        result = feedforward(row)
        cost = compute_cost(row)
        #gradient = compute_gradient(row)
        # update weights of last layer        
        print('\n\ninput', row) #, 'gradient:', gradient)
        print('before update:\n\tw1', w1.get_value(), '\n\tb1', b1.get_value(),'\n\tw2', w2.get_value(), '\n\tb2', b2.get_value(),'\n\tnet out:', result, '\n\tcost', cost)
        update_all(row)
        result_new = feedforward(row)
        cost_new = compute_cost(row)
        print('after update:\n\tw1', w1.get_value(), '\n\tb1', b1.get_value(),'\n\tw2', w2.get_value(), '\n\tb2', b2.get_value(),'\n\tnet out:', result_new, '\n\tcost', cost_new, '\n\tcost decreased?', cost_new < cost)
        
cost_end = compute_avg_cost(data)
print('cost changed from', cost_beginning, 'to', cost_end)

iteration 0


input [1 2 3]
before update:
	w1 [[ 0.36279149  1.52938185]
 [ 0.49476692 -2.12981102]
 [-0.98467004 -0.65122717]] 
	b1 [-0.44553824  0.84295322] 
	w2 [[ 0.1714369   1.51305608 -0.1711786 ]
 [-0.41978625 -1.05248319 -0.72485807]] 
	b2 [ 0.9394343   0.73660137  0.07686146] 
	net out: [ 2.20085049  1.68159287  3.21145924] 
	cost 17.984967860676278
after update:
	w1 [[ 0.36341568  1.53002637]
 [ 0.4960153  -2.12852198]
 [-0.98279747 -0.64929361]] 
	b1 [-0.44491405  0.84359774] 
	w2 [[ 0.17635372  1.51175238 -0.1703128 ]
 [-0.41056139 -1.05492917 -0.72323365]] 
	b2 [ 0.9370326   0.73723818  0.07643855] 
	net out: [ 2.15063252  1.69824905  3.19443778] 
	cost 17.71370283686084 
	cost decreased? True


input [2 3 4]
before update:
	w1 [[ 0.36341568  1.53002637]
 [ 0.4960153  -2.12852198]
 [-0.98279747 -0.64929361]] 
	b1 [-0.44491405  0.84359774] 
	w2 [[ 0.17635372  1.51175238 -0.1703128 ]
 [-0.41056139 -1.05492917 -0.72323365]] 
	b2 [ 0.9370326   0.73723818  0.07643855] 
	net ou

In [24]:
_x = np.asmatrix([1,2,3])
_e = np.asarray([2,4])
#np.random.randn(3,1).shape
_x.transpose() * _e
#_x.transpose() * _e

#_x.dot(_e)
#_e.dot(_x)
?x.reshape

In [107]:
d = np.asarray([[1,2,3],[2,3,4],[5,6,7],[8,9,10]])
d.shape[1]

3